In [1]:
import pandas as pd 
import json
import sqlalchemy


the data structure is a list of tables, each with a JSON object w/ tbl name and rows as a key

Let's parse this into a series of pandas dataframes in a diction w/ the table name as keys


In [5]:
with open('./data/blackcat_dump_may_1.json','r') as d:
    data = json.load(d)

In [6]:
dfs_dict = {}
for table in data['Tables']:
    tbl_name = table['Name']
    try:
        col_list = [k['Column'] for k in table['Rows'][0]['Columns']]
    except IndexError:
        print(f"There are no columns in {tbl_name}")
    rows = []
    for row in table['Rows']:
        values = [d['Value'] for d in row['Columns']]
        rows.append(values)
    dfs_dict[tbl_name] = pd.DataFrame(rows, columns=col_list)

There are no columns in BudgetExpense
There are no columns in BudgetExpenseComments
There are no columns in BudgetExpenseReports
There are no columns in data_Admendment_View
There are no columns in data_BudgetItemsTotals
There are no columns in data_BudgetItemsTotals_Report
There are no columns in data_BudgetItems_Report
There are no columns in data_Budget_Reporting_ServiceData
There are no columns in data_Certification
There are no columns in data_Comment
There are no columns in data_ContractPaymentRequest
There are no columns in data_ContractPaymentRequest_Allocation
There are no columns in data_ContractPaymentRequest_AllocationDistrobution
There are no columns in data_ContractPaymentRequest_CodeStrip
There are no columns in data_ContractPaymentRequest_CodeStripInvoice
There are no columns in data_ContractPaymentRequest_Documents
There are no columns in data_ContractReport5307
There are no columns in data_ContractReports
There are no columns in data_ContractReportsOTPPP
There are no 

In [7]:
print(f"The total number of parsed tables is {len(dfs_dict)}")

The total number of parsed tables is 375


In [8]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///./data/blackcat.sqlite')

In [12]:
# dump to SQLlite
import os
if os.path.exists('./data/blackcat.sqlite'):
    os.remove('./data/blackcat.sqlite')
{v.to_sql(k, con=engine) for k,v in dfs_dict.items()}

{None}

In [13]:
for k,v in dfs_dict.items():
    df = v
    if len(df) == 0:
        display(f"no data in {k}")
    else: 
        display(k)
        display(df.columns)

'ALI'

Index(['ALI', 'Code', 'Description', 'id'], dtype='object')

'ALIGroup'

Index(['ALI Group', 'Code', 'Description'], dtype='object')

'BudgetCategories'

...

'no data in VehicleOrgLink'

'Vehicle_Inventory'

Index(['VehicleId', 'OrganizationId', 'LicensePlate', 'VIN', 'VehicleIDNumber',
       'FundedByDOT', 'FundingSource', 'FundingAgency', 'GrantNumber',
       'StandardAgreementNumber', 'FederalShare', 'FederalSharePercent',
       'TollCredit', 'TollCreditPercent', 'LocalShare', 'LocalSharePercent',
       'TotalCost', 'FTACategory', 'Year', 'Manufacturer', 'Model', 'Chassis',
       'Class', 'FuelType', 'VehicleLength', 'AmbulatorySeatingCapacity',
       'WheelchairPositions', 'WheelchairEquipped', 'VehicleStatus',
       'ImprovementType', 'CurrentCondition', 'CurrentMileage', 'VehicleUse',
       'PurchaseOrderNumber', 'PurchaseOrderDate', 'InvoiceNumber', 'Vendor',
       'ReceivedDate', 'LastInspectionDate', 'NextInspectionDate',
       'StateLienHolder', 'TitleNumber', 'VehicleTitleReleaseDate',
       'VehicleReleaseReportingDate', 'FairMarketValue', 'TransferDate',
       'SoldPrice', 'TransferRecipent', 'TransferStandardAgreementNumber',
       'ReplacementVIN', 'InServiceDat

'_Agency_Contact'

Index(['District', 'Agency', 'RTPA/MPO', 'Title', 'Contact Person',
       'First Name', 'Mid Initial', 'Last Name', 'Role', 'Email', 'Phone',
       'Alt Contact Information', 'F13', 'F14', 'F15'],
      dtype='object')